<a href="https://colab.research.google.com/github/franciscojferrari/Amazing-Landscapes/blob/experiments/experiments_amazing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow as tf
import importlib
import yaml
import os
from google.colab import auth
from pathlib import Path
from tqdm import tqdm 
from PIL import Image

In [2]:
auth.authenticate_user()


In [3]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   112k      0 --:--:-- --:--:-- --:--:--  112k
OK
62 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 62 not upgraded.
Need to get 10.8 MB of archives.
After this operation, 23.1 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.35.0_amd64.deb ...
Unpacking gcsfuse (0.35.0) ...
Setting up gcsfuse (0.35.0) ...


In [4]:
!mkdir dataset
!gcsfuse --implicit-dirs spade_dataset dataset

2021/05/13 09:06:13.723341 Using mount point: /content/dataset
2021/05/13 09:06:13.731595 Opening GCS connection...
2021/05/13 09:06:14.233877 Mounting file system "spade_dataset"...
2021/05/13 09:06:14.265901 File system has been successfully mounted.


In [5]:
PATH = "/content/dataset/generated_images/cityscape/"

Create txt file with the path of the generated images

In [6]:
file_url = list(map(lambda x: PATH + x + '\n', os.listdir(PATH)))

In [7]:
f = open(PATH + "test_images.txt","w")
f.writelines(file_url)
f.close()

Using pre-trained model DRN to create segmentation mask for the generated images

In [8]:
! git clone https://github.com/fyu/drn.git

Cloning into 'drn'...
remote: Enumerating objects: 203, done.
remote: Total 203 (delta 0), reused 0 (delta 0), pack-reused 203
Receiving objects: 100% (203/203), 521.11 KiB | 2.53 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [9]:
import sys
! {sys.executable} -m pip install -r drn/requirements.txt

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! python3 drn/segment.py test -d drn/datasets/cityscapes -l $PATH \
-c 19 --arch drn_d_22 --pretrained /content/gdrive/MyDrive/cityscapes/drn_d_22_cityscapes.pth --phase test --batch-size 1

In [13]:
PATH_gtmask = "/content/dataset/cityscape/gtfine/gtFine/val/"

In [25]:
gt_masks = []
for (dir_path, _, file_names) in os.walk(PATH_gtmask):
  gt_masks.extend([dir_path+ "/" + file_mask for file_mask in file_names if file_mask.endswith("_gtFine_color.png")])
len(gt_masks)

500

In [28]:
generated_masks = [PATH + mask for mask in os.listdir(PATH) if mask.endswith(".png")]
len(generated_masks)

500

In [29]:
file_mask = list(zip(sorted(gt_masks), sorted(generated_masks)))
file_mask[0]

('/content/dataset/cityscape/gtfine/gtFine/val/frankfurt/frankfurt_000000_000294_gtFine_color.png',
 '/content/dataset/generated_images/cityscape/frankfurt_000000_000294..png')